In [251]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import sys
from ipywidgets import widgets
from IPython.display import display, HTML
from IPython.display import clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from tabulate import tabulate
import math

# Customized modules
HOME = '/home/serinatan/project/GPU-Virtualization-Benchmarks/util'
if HOME not in sys.path:
    sys.path.append(HOME)

import data.scripts.common.format as fmt
import data.scripts.common.constants as const
import data.scripts.gen_tables.gen_pair_configs as gen_pair
import data.scripts.gen_tables.search_best_inter as search_inter
import data.scripts.gen_graphs.gen_altair_timeline as gen_altair
import scheduler.scheduler as scheduler
import data.scripts.gen_tables.gen_lut_configs as lut_configs

SCRIPT_PATH = os.path.join(HOME, 'data/scripts') 
PKL_PATH = os.path.join(HOME, 'data/pickles') 
CSV_PATH = os.path.join(HOME, 'data/csv') 
GRAPH_PATH = os.path.join(HOME, 'data/graphs')

%config InlineBackend.figure_format ='retina'
plt.style.use('seaborn-talk')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [252]:
def draw_table(df, cols, hide_index=True):
    if hide_index:
        return df[cols].style.set_table_styles(fmt.table_style).hide_index()
    else:
        return df[cols].style.set_table_styles(fmt.table_style)

# Seq

In [254]:
# run python scripts to generate all the pickles needed
gen_seq = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_seq.py')   
seq_file = os.path.join(CSV_PATH, 'seq-multi.csv')
seq_pkl = os.path.join(PKL_PATH, 'seq-multi.pkl')
%run $gen_seq --multi --csv $seq_file --output $seq_pkl

df_seq = pd.read_pickle(os.path.join(PKL_PATH, 'seq-multi.pkl'))
print(df_seq.columns)

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd', 'ipc',
       'avg_dram_bw', 'avg_dram_eff', 'avg_row_locality', 'std_dram_bw',
       'ratio_dram_bw', 'MPKI', 'l2_access_density', '1_kidx', 'waves'],
      dtype='object')


In [255]:
col_seq = ['pair_str', '1_kidx', 'runtime', 'instructions',
           'ipc',
           'avg_dram_bw', 
           'waves',
           'l1D_miss_rate',
           'l2_miss_rate',
          ]

draw_table(df_seq, col_seq).format({
    'runtime': '{:,}',
    'instructions': '{:,}',
    'avg_dram_bw':'{:.4f}', 
    'sp_busy': '{:.2f}',
    'dp_busy': '{:.2f}',
    'int_busy': '{:.2f}',
    'tensor_busy': '{:.2f}',
    'sfu_busy': '{:.2f}',
})

pair_str,1_kidx,runtime,instructions,ipc,avg_dram_bw,waves,l1D_miss_rate,l2_miss_rate
nvd_conv-0,1,"190,346","405,798,912",2131.9,0.7242,7.2,0.9991,0.931
nvd_conv-0,2,"208,164","365,518,848",1755.92,0.6698,14.4,0.9997,0.9323
parb_mriq-0,1,"1,286","45,056",35.0358,0.0005,0.0125,1,0.3292
parb_mriq-0,2,"3,157,989","21,735,183,780",6882.6,0.0019,1.6,0.7143,0.3666
parb_sad-0,1,"51,633","138,709,296",2686.45,0.0061,1.16471,1,0.6109
parb_sad-0,2,"21,556","4,380,156",203.199,0.4558,0.0773438,0.9099,0.9849
parb_sad-0,3,"12,806","1,151,370",89.9086,0.1249,0.0386719,0.9109,0.9371
parb_sad-1,1,"3,000,706","11,264,876,160",3754.08,0.1736,94.5882,0.9999,0.5524
parb_sad-1,2,"2,324,970","355,721,760",153.001,0.5645,6.28125,0.9121,0.9998
parb_sad-1,3,"541,865","93,505,200",172.562,0.5842,3.14062,0.912,0.9992


# Intra

In [263]:
gen_intra = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_intra.py')
intra_file = os.path.join(CSV_PATH, 'intra-multi.csv')
intra_pkl = os.path.join(PKL_PATH, 'intra-multi.pkl')

%run $gen_intra --out_intra $intra_pkl --seq $seq_pkl --csv $intra_file

df_intra = pd.read_pickle(intra_pkl)
df_intra.sort_values(['pair_str', '1_kidx'], inplace=True)
# col_intra = ['pair_str', '1_kidx', 'regs', 'thread_count', 'smem', 'intra']
# draw_table(df_intra, col_intra)

In [264]:
col_prod = ['pair_str_x', '1_kidx_x', 'pair_str_y', '1_kidx_y',
            'norm_ipc_x', 'norm_ipc_y', 'diff_mflat', 'sum_ipc', 
            'intra_x', 'intra_y', 
           'sum_comp', 'sum_dram',]
find_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_pair_configs.py')
pairs = df_seq.apply(lambda row: ':'.join([row['pair_str'], str(row['1_kidx'])]), axis=1)

In [265]:
app1 = widgets.Dropdown(options=pairs, value=pairs[0], description='App 1:')
app2 = widgets.Dropdown(options=pairs, value=pairs[1], description='App 2:')

qos = widgets.FloatSlider(
    value=0.75,
    min=0.1,
    max=0.95,
    step=0.05,
    description='QoS:',
    readout_format='.2f',
)

button = widgets.Button(description='Calculate', button_style='info')

def show_widgets_intra():
    display(app1)
    display(app2)
    display(qos)
    display(button)
    

def onclick(b):
    clear_output()
    show_widgets_intra()
    %run $find_pair --app $app1.value $app2.value --qos $qos.value --intra_pkl $intra_pkl --top
    
    df_prod = pd.read_pickle(os.path.join(PKL_PATH, 'pair_candidates.pkl'))
   
    display(HTML(draw_table(df_prod, col_prod, False).render()))
    
    

In [266]:
show_widgets_intra()
button.on_click(onclick)

Dropdown(description='App 1:', index=8, options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_mriq-0:1', 'parb_mriq-0…

Dropdown(description='App 2:', index=9, options=('nvd_conv-0:1', 'nvd_conv-0:2', 'parb_mriq-0:1', 'parb_mriq-0…

FloatSlider(value=0.75, description='QoS:', max=0.95, min=0.1, step=0.05)

Button(button_style='info', description='Calculate', style=ButtonStyle())

# Pair-Dynamic

In [267]:
gen_pair = os.path.join(SCRIPT_PATH, 'gen_tables/gen_table_pair.py')
dynamic_csv = os.path.join(CSV_PATH, 'pair_dynamic_multi.csv')
dynamic_output = os.path.join(PKL_PATH, 'pair_dynamic_multi.pkl')
%run $gen_pair --csv $dynamic_csv --output $dynamic_output --seq_pkl $seq_pkl --qos 0.5 --how dynamic --multi --isolated_pkl $intra_pkl

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd'],
      dtype='object')


In [312]:
df_dynamic = pd.read_pickle(dynamic_output)
col_dynamic = ['1_bench', '1_kidx', '1_intra', '2_bench', '2_kidx', '2_intra', 'sld', 'ws', 
               'importance', 'weighted_increase', 'sum_increase']

df_dynamic_show = df_dynamic[(df_dynamic['1_bench'] == 'parb_mriq-0') & (df_dynamic['2_bench'] == 'parb_sad-1')].sort_values(
    '2_kidx')
df_dynamic_show[col_dynamic]

,1_bench,1_kidx,1_intra,2_bench,2_kidx,2_intra,sld,ws,importance,weighted_increase,sum_increase
0,parb_mriq-0,1,1.0,parb_sad-1,1,12.0,"[0, 1.0016999624961134, 1.0648958067172019]",2.066596,"[0, 0.00040705541619517134, 0.5114077600821196]","[0, 0.00040636461159571097, 0.48024206392422314]",0.480648
1,parb_mriq-0,2,3.0,parb_sad-1,1,8.0,"[0, 0.5870033874562999, 0.7143036356880208]",1.301307,"[0, 0.9995929445838049, 0.5114077600821196]","[0, 1.7028742353862834, 0.7159529008830106]",2.418827
2,parb_mriq-0,2,5.0,parb_sad-1,1,8.0,"[0, 0.6730921830450515, 0.6709803533982301]",1.344073,"[0, 0.9995929445838049, 0.5114077600821196]","[0, 1.4850758480980606, 0.7621799319340079]",2.247256
3,parb_mriq-0,2,2.0,parb_sad-1,1,8.0,"[0, 0.48414949141129915, 0.8875987554060534]",1.371748,"[0, 0.9995929445838049, 0.5114077600821196]","[0, 2.064636981585965, 0.5761699833030567]",2.640807
15,parb_mriq-0,2,4.0,parb_sad-1,1,8.0,"[0, 0.6382826619754007, 0.683548623269719]",1.321831,"[0, 0.9995929445838049, 0.5114077600821196]","[0, 1.5660662652032509, 0.7481658841412443]",2.314232
6,parb_mriq-0,1,1.0,parb_sad-1,1,4.0,"[0, 1.125569349936965, 0.6720608510947087]",1.797630,"[0, 0.00040705541619517134, 0.5114077600821196]","[0, 0.00036164401262167237, 0.7609545463764124]",0.761316
16,parb_mriq-0,1,1.0,parb_sad-1,1,8.0,"[0, 1.020659121206852, 0.9848034630670723]",2.005463,"[0, 0.00040705541619517134, 0.5114077600821196]","[0, 0.0003988162234947346, 0.5192993112446935]",0.519698
11,parb_mriq-0,1,1.0,parb_sad-1,1,16.0,"[0, 0.3579681649137337, 0.9289475859391664]",1.286916,"[0, 0.00040705541619517134, 0.5114077600821196]","[0, 0.0011371274210746287, 0.5505238054578571]",0.551661
5,parb_mriq-0,1,1.0,parb_sad-1,2,2.0,"[0, 0.24119155959384944, 1.3394713600044246]",1.580663,"[0, 0.00040705541619517134, 0.3962426508822009]","[0, 0.0016876851614568338, 0.2958201740729208]",0.297508
7,parb_mriq-0,2,7.0,parb_sad-1,2,2.0,"[0, 0.9603730688314366, 1.3359792768265226]",2.296352,"[0, 0.9995929445838049, 0.3962426508822009]","[0, 1.040838166984514, 0.2965934111069697]",1.337432


In [269]:
df_seq_index = df_seq.set_index(['pair_str', '1_kidx'])
df_intra_index = df_intra.set_index(['pair_str', '1_kidx', 'intra'])

In [319]:
def show_widgets_best():
    display(best_dd)
    display(best_type_dd)
    display(best_btn)
    

def best_onclick(b):
    clear_output()
    show_widgets_best()
    
    # apps = best_dd.value.split('+')
    apps = re.split(r'-(?=\D)', best_dd.value)
           
    # Output LUT resource configuration 
    weighted = (best_type_dd.value == 'Global (weighted)')
    
    configs, interference = lut_configs.get_lut_matrix(apps, df_dynamic, weighted)

    headers = [const.gen_kernel_headers(apps[0]), const.gen_kernel_headers(apps[1])]
    
    lut_configs.pretty_print_matrix(apps, 'configs', configs, headers)
    lut_configs.pretty_print_matrix(apps, 'slowdown', interference, headers)
    print('\n')
   
    lut_configs.print_rsrc_usage(configs, headers, df_intra_index)
    print('\n')
    
    # Run simulation to predict final slowdown
    scaled_runtime, norm_ipc = lut_configs.predict_app(apps, interference, df_seq_index)
    
    row = pd.Series({'1_bench': apps[0], '2_bench': apps[1], 
                     'runtime': scaled_runtime, 'norm_ipc': norm_ipc})
    
    gen_altair.draw_altair_timeline(row, title='Prediction').display()  


best_dd = widgets.Dropdown(options=df_dynamic['pair_str'].unique(), description='Pair:')
best_type_dd = widgets.Dropdown(options=['Local', 'Global (weighted)'], value='Local', description='Type')
best_btn = widgets.Button(description='Best', button_style='info')


show_widgets_best()
best_btn.on_click(best_onclick)


Dropdown(description='Pair:', options=('parb_mriq-0-parb_sad-1', 'parb_sad-0-parb_sad-1', 'nvd_conv-0-parb_sad…

Dropdown(description='Type', options=('Local', 'Global (weighted)'), value='Local')

Button(button_style='info', description='Best', style=ButtonStyle())

parb_mriq-0 configs
              parb_mriq-0:1  parb_mriq-0:2  parb_mriq-0:3
parb_sad-1:1              1              2              2
parb_sad-1:2              1              7              7
parb_sad-1:3              1              7              7
--------------------------------------------------
parb_sad-1 configs
              parb_mriq-0:1  parb_mriq-0:2  parb_mriq-0:3
parb_sad-1:1             12              8              8
parb_sad-1:2              2              2              2
parb_sad-1:3              4              4              4
--------------------------------------------------
parb_mriq-0 slowdown
              parb_mriq-0:1  parb_mriq-0:2  parb_mriq-0:3
parb_sad-1:1       1.001700       0.484149       0.484149
parb_sad-1:2       0.241192       0.960373       0.960373
parb_sad-1:3       0.425124       0.985649       0.985649
--------------------------------------------------
parb_sad-1 slowdown
              parb_mriq-0:1  parb_mriq-0:2  parb_mriq-0:3
parb_sad-1:1 

alt.LayerChart(...)

# CTX Ratio

In [303]:
ctx_csv = os.path.join(HOME, 'data/csv/ctx.csv')
output = os.path.join(PKL_PATH, 'pair_ctx.pkl')


In [304]:
%run $gen_pair --csv $ctx_csv --output $output --seq_pkl $seq_pkl --qos 0.5 --multi --how ctx

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd'],
      dtype='object')


In [305]:
df_ctx= pd.read_pickle(output)
uniq_pairs = df_ctx[['1_bench', '2_bench']].drop_duplicates().values
uniq_pairs = ['+'.join(x) for x in uniq_pairs]

col_ctx = ['1_bench', '2_bench', '1_ctx', '2_ctx', 
           'cta_quota',
#            'runtime', 'norm_runtime'
          ]
draw_table(df_ctx, col_ctx)

1_bench,2_bench,1_ctx,2_ctx,cta_quota
parb_mriq-0,parb_sad-1,0.5,0.5,"[[], [1, 4, 4], [8, 8, 16]]"
parb_mriq-0,parb_sad-1,0.25,0.75,"[[], [1, 2, 2], [12, 12, 24]]"
parb_mriq-0,parb_sad-1,0.75,0.25,"[[], [1, 6, 6], [4, 4, 8]]"
parb_sad-0,parb_sad-1,0.75,0.25,"[[], [12, 2, 2], [4, 4, 8]]"
parb_sad-0,parb_sad-1,0.25,0.75,"[[], [4, 2, 2], [12, 12, 24]]"
parb_sad-0,parb_sad-1,0.5,0.5,"[[], [8, 2, 2], [8, 8, 16]]"
nvd_conv-0,parb_sad-1,0.75,0.25,"[[], [24, 12], [4, 4, 8]]"
nvd_conv-0,parb_sad-1,0.25,0.75,"[[], [8, 4], [12, 12, 24]]"
nvd_conv-0,parb_sad-1,0.5,0.5,"[[], [16, 8], [8, 8, 16]]"
parb_mriq-0,parb_sad-0,0.5,0.5,"[[], [1, 4, 4], [8, 2, 2]]"


In [314]:
df_dynamic_index = df_dynamic.set_index(['1_bench', '1_kidx', '1_intra', '2_bench', '2_kidx', '2_intra'])

In [320]:
ctx_dd = widgets.Dropdown(options=uniq_pairs, value=uniq_pairs[0], description='Pair:')
ctx_btn = widgets.Button(description='Display', button_style='info')

def show_widgets_ctx():
    display(ctx_dd)
    display(ctx_btn)
    

def ctx_onclick(b):
    clear_output()
    show_widgets_ctx()
    
    apps = ctx_dd.value.split('+')
    df_apps = df_ctx[(df_ctx['1_bench'] == apps[0]) & (df_ctx['2_bench'] == apps[1])]
    df_apps = df_apps.sort_values('1_ctx')
    for rid, row in df_apps.iterrows():
        # Show actual timeline
        gen_altair.draw_altair_timeline(row, col_title='1_ctx').display()  
        # Print ctas/SM for each kernel in each app
        print(apps[0], ':', row['cta_quota'][1], 'CTAs/SM')
        print(apps[1], ':', row['cta_quota'][2], 'CTAs/SM')
        
        print('slowdown:', row['sld'])
        print('weighted speedup:', row['ws'])
        
        interference = lut_configs.get_ctx_matrix(apps, row, df_dynamic_index)
        
        if interference:
            headers = [const.gen_kernel_headers(apps[0]), const.gen_kernel_headers(apps[1])]
            lut_configs.pretty_print_matrix(apps, 'interference', interference, headers)

            # Predict app slowdown
            scaled_runtime, norm_ipc = lut_configs.predict_app(apps, interference, df_seq_index)
            row = pd.Series({'1_bench': apps[0], '2_bench': apps[1], 
                     'runtime': scaled_runtime, 'norm_ipc': norm_ipc})
    
            gen_altair.draw_altair_timeline(row, title='Prediction').display()  

        print('-' * 100)

show_widgets_ctx()
ctx_btn.on_click(ctx_onclick)
 

Dropdown(description='Pair:', options=('parb_mriq-0+parb_sad-1', 'parb_sad-0+parb_sad-1', 'nvd_conv-0+parb_sad…

Button(button_style='info', description='Display', style=ButtonStyle())

alt.LayerChart(...)

parb_mriq-0 : [1, 2, 2] CTAs/SM
parb_sad-1 : [12, 12, 24] CTAs/SM
slowdown: [0, 0.5166694801362505, 0.9239983408302526]
weighted speedup: 1.4406678209665031
----------------------------------------------------------------------------------------------------


alt.LayerChart(...)

parb_mriq-0 : [1, 4, 4] CTAs/SM
parb_sad-1 : [8, 8, 16] CTAs/SM
slowdown: [0, 0.6995541179447456, 0.8355819718343556]
weighted speedup: 1.535136089779101
----------------------------------------------------------------------------------------------------


alt.LayerChart(...)

parb_mriq-0 : [1, 6, 6] CTAs/SM
parb_sad-1 : [4, 4, 8] CTAs/SM
slowdown: [0, 0.8048443053816315, 0.6127944956974137]
weighted speedup: 1.4176388010790453
----------------------------------------------------------------------------------------------------


# CTA LUT

In [112]:
lut_csv = os.path.join(HOME, 'data/csv/pair_lut.csv')
lut_out = os.path.join(PKL_PATH, 'pair_lut.pkl')

%run $gen_pair --csv $lut_csv --output $lut_out --seq_pkl $seq_pkl --multi --how lut

Index(['pair_str', 'config', 'gpusim_version', 'jobId', 'stall_icnt_to_l2',
       'stall_l2_to_icnt', 'stall_core_ldst', 'l1D_miss_rate', 'l2_miss_rate',
       'l2_rshr_entry_fail', 'l2_rshr_merge_fail', 'l2_total_accesses',
       'mem_count', 'empty_warp', 'stall_warp', 'idle_warp', 'scoreboard_warp',
       'tot_warp_insn', 'runtime', 'instructions', 'l2_bw', 'avg_mem_lat',
       'avg_core_to_l2', 'avg_l2_to_core', 'avg_mrq_latency', 'dram_eff',
       'dram_bw', 'row_buffer_locality', 'mem_idle', 'total_cmd'],
      dtype='object')


In [113]:
df_lut = pd.read_pickle(lut_out)
col_lut = ['pair_str', '1_bench', '2_bench', 'norm_ipc', 'runtime'] 
# draw_table(df_lut, col_lut)

In [114]:
lut_dd = widgets.Dropdown(options=df_lut['pair_str'], value=df_lut.iloc[0]['pair_str'], description='Pair:')

lut_btn = widgets.Button(description='Display', button_style='info')

def show_widgets_lut():
    display(lut_dd)
    display(lut_btn)
    

def lut_onclick(b):
    clear_output()
    show_widgets_lut()
    
    apps = re.split(r'-(?=\D)', lut_dd.value)
    df_apps = df_lut[(df_lut['1_bench'] == apps[0]) & (df_lut['2_bench'] == apps[1])]
    for rid, row in df_apps.iterrows():
        gen_altair.draw_altair_timeline(row, title='CTA LUT').display()  
        
        print('Note:')
#         print(row['1_bench'], ':', row['cta_quota'][1], 'CTAs/SM')
#         print(row['2_bench'], ':', row['cta_quota'][2], 'CTAs/SM')
        
        print('runtime:', row['runtime'])
        print('slowdown:', row['sld'])
        print('weighted speedup:', row['ws'])
        print('-' * 100)

show_widgets_lut()
lut_btn.on_click(lut_onclick)
 

Dropdown(description='Pair:', index=1, options=('parb_sad-0-nvd_conv-0', 'parb_sad-1-nvd_conv-0'), value='parb…

Button(button_style='info', description='Display', style=ButtonStyle())

alt.LayerChart(...)

Note:
runtime: [[], [3903782, 3114421, 816561], [251500, 231423, 247800, 224266, 250280, 226474, 246663, 226125, 245418, 225838, 246431, 225143, 251086, 227849, 248230, 226570, 327024, 436394, 409356, 415062, 372055, 419565, 384013, 431129, 323182, 409296, 221001, 206416]]
slowdown: [0, 0.7489110074023927, 0.6703564568697488]
weighted speedup: 1.4192674642721415
----------------------------------------------------------------------------------------------------
